## 爬取豆瓣电影Top250排行榜各电影信息

In [1]:
import json
import requests
import os
from tqdm import tqdm_notebook
import operator

In [2]:
#新建文件夹
if not os.path.exists('top250'):
    os.mkdir('top250')

In [3]:
#爬取全部250部电影
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
for i in tqdm_notebook(range(25)):
    url = f'https://api.douban.com/v2/movie/top250?apikey={access_key}&start={i*10}&count={10}'
    
    json_file_path = os.path.join('top250', '{}.json'.format(i))
    if os.path.exists(json_file_path):  #若已爬过，跳过
        continue  
        
    #防止超时，访问接口直到访问成功
    while True:
        try:
            response = requests.get(url,headers = headers,timeout = 5)
            break
        except requests.exceptions.Timeout:
            time.sleep(1)
            
    #写入文件
    with open(json_file_path,'w',encoding='utf-8') as f:
        f.write(json.dumps(json.loads(response.content.decode('utf-8')),indent=4,ensure_ascii=False))
        #indent=4:对齐 ensure_ascii=False:编码中文 json.dumps()用于将字典形式的数据转化为字符串，json.loads()用于将字符串形式的数据转化为字典

In [4]:
#将上面取的文件合并为top250.json
movies = []
for json_file in tqdm_notebook(os.listdir('top250')):
    json_file_path = os.path.join('top250', json_file)
    with open(json_file_path,'r',encoding = 'utf8') as f:
        
        movies += json.load(f)['subjects']

#按平均分降序排序
movies = sorted(movies,
               key = lambda movie: movie['rating']['average'],reverse= True)

#写入文件
with open('top250.json','w',encoding = 'utf8') as f:
    f.write(json.dumps({'subjects':movies},indent=4,ensure_ascii=False))

## TOP 250 电影数据可视化分析

In [7]:
#加载数据
with open('top250.json',encoding='utf8') as f:
    top250_movies = json.load(f)['subjects']

top250_movies[0]  # 排名第一的电影

{'rating': {'max': 10,
  'average': 9.7,
  'details': {'1': 1407.0,
   '3': 18900.0,
   '2': 1166.0,
   '5': 1158180.0,
   '4': 186041.0},
  'stars': '50',
  'min': 0},
 'genres': ['犯罪', '剧情'],
 'title': '肖申克的救赎',
 'casts': [{'avatars': {'small': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p17525.jpg',
    'large': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p17525.jpg',
    'medium': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p17525.jpg'},
   'name_en': 'Tim Robbins',
   'name': '蒂姆·罗宾斯',
   'alt': 'https://movie.douban.com/celebrity/1054521/',
   'id': '1054521'},
  {'avatars': {'small': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p34642.jpg',
    'large': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p34642.jpg',
    'medium': 'https://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p34642.jpg'},
   'name_en': 'Morgan Freeman',
   'name': '摩根·弗里曼',
   'alt': 'h

#### 年份和评分的变化

In [9]:
import numpy as np

# 平均评分 list
average_rating_array = np.array(
    [x['rating']['average'] for x in top250_movies])

# 发行年 list
year_array = np.array([x['year'] for x in top250_movies]).astype(int)

# 去重
unique_year_array = np.unique(year_array)
rating_y = []
number_y = []

# 统计一年的 TOP 250 平均评分，发行电影数量
for i in range(unique_year_array.shape[0]):
    year = unique_year_array[i]
    rating_y.append(np.average(average_rating_array[year_array == year]))
    number_y.append(np.sum(year_array == year))

len(rating_y), len(number_y)

(55, 55)

###### 通过 Plotly 绘制出折现统计图

In [12]:
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

init_notebook_mode(connected=True)
# 绘图
trace1 = go.Scatter(
    x=unique_year_array,
    y=rating_y,
    name='年度平均评分',
)
trace2 = go.Scatter(
    x=unique_year_array,
    y=number_y,
    name='年度电影数量'
)
data = [trace1, trace2]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='豆瓣 TOP 250 条形折线图')

结论：优秀电影产量在1983年后开始激增
优秀电影最早诞生于1931年

In [20]:
#查看1931年的电影
[x['title'] for x in top250_movies if int(x['year']) == 1931]

['城市之光']

1931年的优秀作品为喜剧大师 查理·卓别林 的代表作之一《城市之光》

1994、2001、2004 和 2010 是最为高产的几年，这几年的 TOP 250 电影最多，表明在这几年的电影质量非常好。

2010 年的 TOP 250 的电影虽然数量多，但是平均评分并不高，相反 1994 数量多的同时，评分也都达到了 9.0。所以来看一下 1994 年的这 11 部电影到底是哪些？

In [22]:
[x['title'] for x in top250_movies if int(x['year']) == 1994]

['肖申克的救赎',
 '阿甘正传',
 '这个杀手不太冷',
 '活着',
 '饮食男女',
 '狮子王',
 '燃情岁月',
 '低俗小说',
 '阳光灿烂的日子',
 '重庆森林',
 '东邪西毒']

#### TOP 250 电影类型

In [23]:
import itertools

# 统计出现的类型，一个电影可能对应多个标签
genres = list(itertools.chain(*[x['genres'] for x in top250_movies]))
labels = list(set(genres))

# 统计标签出现的次数
values = [genres.count(label) for label in labels]

# 绘图
trace = go.Pie(labels=labels, values=values)
iplot([trace], filename='豆瓣 TOP 250 电影分类百分比')

统计结果表明在 TOP 250 电影里 剧情 出现的次数最多，占到了 30.7%, 其次则是爱情、喜剧等。

In [25]:
#查看纪录片
[x['title'] for x in top250_movies if '纪录片' in x['genres']]

['人生果实', '海豚湾', '海洋', '二十二']